# RESUME CATEGORIZATION

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df=pd.read_csv('UpdatedResumeDataSet.csv')

Category


In [ ]:
df['Category'].value_counts()

In [ ]:
plt.figure(figsize=(15, 5))
sns.countplot(df['Category'])
plt.show()

In [ ]:
counts =  df['Category'].value_counts()
label = df['Category'].unique()
plt.figure(figsize=(15, 10))
plt.pie(counts, labels=label, autopct='%1.1f%%', shadow=True, colors=plt.cm.plasma(np.linspace(0,1,3)))
plt.show()

In [ ]:
df['Category'].unique()

Cleaning of data

In [ ]:
import re
def cleanResume(txt):
    cleanText = re.sub('http\S+\s', ' ', txt)
    cleanText = re.sub('RT|cc', ' ', cleanText)
    cleanText = re.sub('#\S+\s', ' ', cleanText)
    cleanText = re.sub('@\S+', '  ', cleanText)  
    cleanText = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', cleanText)
    cleanText = re.sub(r'[^\x00-\x7f]', ' ', cleanText) 
    cleanText = re.sub('\s+', ' ', cleanText)
    return cleanText

In [ ]:
df['Resume']=df['Resume'].apply(lambda x: cleanResume(x))

In [ ]:
df['Resume']

words into categorical value

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [ ]:
import pickle
le.fit(df['Category'])
df['Category'] = le.transform(df['Category'])
pickle.dump(le, open('label_encoder.pkl', 'wb'))

In [ ]:
df.Category.unique()

vector

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
tfidf.fit(df['Resume'])
requiredText=tfidf.transform(df['Resume'])

splitting

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(requiredText, df['Category'], test_size=0.2, random_state=42)

model training

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score

clf= OneVsRestClassifier(KNeighborsClassifier())
clf.fit(x_train, y_train)
ypred=clf.predict(x_test)
print(accuracy_score(y_test, ypred))

prediction system

In [ ]:
import pickle
pickle.dump(tfidf, open('tfidf.pkl', 'wb'))
pickle.dump(clf, open('clf.pkl', 'wb'))

In [ ]:
myresume = """I am a data scientist specializing in machine
learning, deep learning, and computer vision. With
a strong background in mathematics, statistics,
and programming, I am passionate about
uncovering hidden patterns and insights in data.
I have extensive experience in developing
predictive models, implementing deep learning
algorithms, and designing computer vision
systems. My technical skills include proficiency in
Python, Sklearn, TensorFlow, and PyTorch.
What sets me apart is my ability to effectively
communicate complex concepts to diverse
audiences. I excel in translating technical insights
into actionable recommendations that drive
informed decision-making.
If you're looking for a dedicated and versatile data
scientist to collaborate on impactful projects, I am
eager to contribute my expertise. Let's harness the
power of data together to unlock new possibilities
and shape a better future.
Contact & Sources
Email: 611noorsaeed@gmail.com
Phone: 03442826192
Github: https://github.com/611noorsaeed
Linkdin: https://www.linkedin.com/in/noor-saeed654a23263/
Blogs: https://medium.com/@611noorsaeed
Youtube: Artificial Intelligence
ABOUT ME
WORK EXPERIENCE
SKILLES
NOOR SAEED
LANGUAGES
English
Urdu
Hindi
I am a versatile data scientist with expertise in a wide
range of projects, including machine learning,
recommendation systems, deep learning, and computer
vision. Throughout my career, I have successfully
developed and deployed various machine learning models
to solve complex problems and drive data-driven
decision-making
Machine Learnine
Deep Learning
Computer Vision
Recommendation Systems
Data Visualization
Programming Languages (Python, SQL)
Data Preprocessing and Feature Engineering
Model Evaluation and Deployment
Statistical Analysis
Communication and Collaboration
"""

In [ ]:
import pickle
clf=pickle.load(open('clf.pkl', 'rb'))
cleaned_resume=cleanResume(myresume)
input_features=tfidf.transform([cleaned_resume])

prediction_id=clf.predict(input_features)[0]
print(prediction_id)

# RESUME ATS SCORE PREDICTION

In [ ]:
%pip install -U sentence-transformers

In [15]:
# === Load Kaggle ATS Dataset and Compute ATS Scores ===

import pickle
from sentence_transformers import SentenceTransformer, util

# Step 1: Load the parsed resume data (as tuple list)
with open('E:/FINAL_YEAR_PROJECT/Resume Screening/backend/ats/train_data.pkl', 'rb') as f:
    kaggle_data = pickle.load(f)

# Step 2: Check structure
print("📄 Sample Entry Type:", type(kaggle_data[0]))
print("📄 Sample Entry Content:", kaggle_data[0])

# Step 3: Convert each tuple into resume text
resume_texts = []
for tup in kaggle_data:
    # Safely extract each component
    summary     = tup[0] if len(tup) > 0 else ""
    skills      = tup[1] if len(tup) > 1 else ""
    experience  = tup[2] if len(tup) > 2 else ""
    designation = tup[3] if len(tup) > 3 else ""

    # Format resume text
    resume = f"""
    Skills: {skills}
    Summary: {summary}
    Experience: {experience}
    Designation: {designation}
    """
    resume_texts.append(resume.strip())

print(f"\n✅ Loaded {len(resume_texts)} resumes.")

# Step 4: Define Sample Job Descriptions
job_descriptions = [
    "We are looking for a data scientist skilled in Python, machine learning, and data visualization.",
    "Frontend React developer needed with strong UI/UX skills and JavaScript expertise."
]

# Step 5: Load Semantic Similarity Model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Step 6: Calculate and Display ATS Match Scores
print("\n🔍 Calculating ATS Match Scores...\n")
for i, resume in enumerate(resume_texts[:3]):  # Show only first 3 for demo
    print(f"📄 Resume {i+1}")
    for j, jd in enumerate(job_descriptions):
        score = util.cos_sim(
            model.encode(resume, convert_to_tensor=True),
            model.encode(jd, convert_to_tensor=True)
        )[0][0]
        print(f"   🔗 JD {j+1} ATS Score: {round(float(score) * 100, 2)}%")
    print("-" * 40)


📄 Sample Entry Type: <class 'tuple'>
📄 Sample Entry Content: ('Govardhana K Senior Software Engineer  Bengaluru, Karnataka, Karnataka - Email me on Indeed: indeed.com/r/Govardhana-K/ b2de315d95905b68  Total IT experience 5 Years 6 Months Cloud Lending Solutions INC 4 Month • Salesforce Developer Oracle 5 Years 2 Month • Core Java Developer Languages Core Java, Go Lang Oracle PL-SQL programming, Sales Force Developer with APEX.  Designations & Promotions  Willing to relocate: Anywhere  WORK EXPERIENCE  Senior Software Engineer  Cloud Lending Solutions -  Bangalore, Karnataka -  January 2018 to Present  Present  Senior Consultant  Oracle -  Bangalore, Karnataka -  November 2016 to December 2017  Staff Consultant  Oracle -  Bangalore, Karnataka -  January 2014 to October 2016  Associate Consultant  Oracle -  Bangalore, Karnataka -  November 2012 to December 2013  EDUCATION  B.E in Computer Science Engineering  Adithya Institute of Technology -  Tamil Nadu  September 2008 to June 2012  htt

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]


🔍 Calculating ATS Match Scores...

📄 Resume 1
   🔗 JD 1 ATS Score: 29.04%
   🔗 JD 2 ATS Score: 13.53%
----------------------------------------
📄 Resume 2
   🔗 JD 1 ATS Score: 27.23%
   🔗 JD 2 ATS Score: 13.06%
----------------------------------------
📄 Resume 3
   🔗 JD 1 ATS Score: 35.58%
   🔗 JD 2 ATS Score: 19.05%
----------------------------------------
